# Import Dependensi

Package

In [20]:
import pandas as pd
import numpy as np

Modul Perhitungan ET

In [21]:
from modul.gravimetri import gravimetri_numpy as gravimetri
from modul.bc_model import bc_model
from modul.hs_model import hs_model
from modul.pm_model import pm_model

Variabel

In [22]:
from variabel import variabel as var

# Import Dataset

In [23]:
# Import file excel
dataset_primer = 'data/data_processed.xlsx'
dataset_sekunder = 'data/dataset_sekunder_2.xlsx'

# Membaca semua sheet dari file Excel
data_primer = pd.ExcelFile(dataset_primer)
data_sekunder = pd.ExcelFile(dataset_sekunder)

# Fungsi-Fungsi

In [24]:
# Menghitung luas area di dalam variabel
def hitung_luas_media(sisi):
        cm_to_mm = sisi[0] * 10
        luas = (cm_to_mm ** 2)
        return luas              # Hasil dalam satuan mm

In [25]:
# Menghitung tanggal pengambilan data
def hitung_tanggal_data(day):
    day = int(day.split()[1])

    tahun = '2024'

    if day > 10:
        bulan = '11'
    else:
        bulan = '10'
    
    def hitung_tanggal(input_day):
        # Jika x antara 1 dan 31, y adalah x + 20, tetapi melingkar kembali ke awal setelah 31
        return (input_day + 21) % 31 if (input_day + 21) % 31 != 0 else 31

    hari = hitung_tanggal(day)
    hari = f"{hari:02d}"

    tanggal = (f"{tahun}-{bulan}-{hari}")

    return tanggal

# ET Model Gravimetri/Penimbangan

Perhitungan Gravimetri

In [26]:
hasil_gravimetri = []

for sheet_name in data_primer.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_primer = data_primer.parse(sheet_name)
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    # Logika program Anda
    for kode in var.sisi_media_tanaman.keys():
        kode_berat = f'berat_{kode}'
        kode_irigasi = f'supply_air_{kode}'
        
        # Buat array untuk variabel yang akan digunakan
        waktu = df_data_primer['waktu'].array
        berat = df_data_primer[kode_berat].array                            # Satuan kg
        irigasi_liter = df_data_primer[kode_irigasi].fillna(0).array        # Satuan liter
        curah_hujan = df_data_sekunder['curah_hujan'].array                 # Satuan mm

        # Hitung luas alas media tanaman
        luas_media = hitung_luas_media(var.sisi_media_tanaman[kode])    # Satuan mm^2

        # Ubah irigasi ke satuan mm
        irigasi_mm = (irigasi_liter * 1e6) / (luas_media)               # Satuan mm

        # Ubah berat ke satuan volume mm^3
        volume = berat * 1e6                                            # Satuan mm^3

        # Hitung total supply air
        supply_air = irigasi_mm + curah_hujan                           # Satuan mm

        perubahan_berat, perubahan_et, et_total = evapotranspirasi = gravimetri(berat=volume,
                                                                                waktu=waktu,
                                                                                luas=luas_media,
                                                                                supply_air=supply_air
                                                                                )

        # Simpan hasil ke list
        hasil_gravimetri.append({
            "Num Day": sheet_name,
            "Kode": kode,
            "ET Total": et_total
        })

# Buat DataFrame dari hasil
df_hasil_gravimetri = pd.DataFrame(hasil_gravimetri)

# Pivot tabel dengan "Day" sebagai index dan "Kode" sebagai kolom
df_hasil_gravimetri = df_hasil_gravimetri.pivot(index="Num Day", columns="Kode", values="ET Total")


In [27]:
df_hasil_gravimetri.to_csv('hasil/ET Aktual Gravimetri.csv', index=True)
df_hasil_gravimetri.head()

Kode,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
Num Day,,,,,,,,,,,,,,,,,,,,,
day 10,11.094,9.046,179.018,11.862,15.572222,206.494444,13.838889,11.705556,10.664286,8.329592,...,8.31250,9.6750,6.164815,8.870988,5.463580,4.594444,0.870,4.414,7.646,9.782
day 12,11.396,10.948,12.388,13.316,14.144444,15.788889,14.944444,12.544444,12.622449,12.948980,...,11.35000,11.2500,13.532099,13.779012,12.820988,11.003704,7.068,3.300,8.084,8.428
day 13,12.146,13.298,13.202,14.130,18.187778,19.365556,7.210000,15.921111,15.344694,0.014082,...,12.86000,12.8350,15.785309,11.953210,10.975432,11.844568,8.618,6.618,9.210,8.154
day 14,13.168,13.232,12.016,11.760,16.786667,16.208889,14.320000,13.453333,12.442449,12.817959,...,11.63250,1.4325,13.361975,11.929877,11.791605,13.243457,4.360,7.032,8.224,8.272
day 15,9.088,8.592,10.560,9.520,12.533333,11.955556,13.122222,7.000000,11.248980,12.236735,...,6.71875,6.5875,9.911111,6.627160,7.501235,8.098765,0.612,6.408,2.804,2.124


# ET Model Blaney-Criddle

In [28]:
hasil_bc_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)

    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")
    
    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    # print(f"Suhu udara {sheet_name}: {suhu_udara}")

    # Menghitung rata-rata suhu udara
    mean_suhu_udara = np.mean(suhu_udara)
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Blaney-Criddle
    et_model_bc = bc_model(var.persentase_hari_terang, mean_suhu_udara)
    # print(f"ET Blaney-Criddle: {et_model_bc}\n")

    # Simpan hasil ke list
    hasil_bc_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Mean Suhu Udara": mean_suhu_udara,
        "ET Total": et_model_bc
    })

# Buat DataFrame dari hasil
df_hasil_bc = pd.DataFrame(hasil_bc_model)

In [29]:
df_hasil_bc.to_csv('hasil/Estimasi ET Blaney-Criddle.csv', index=False)
df_hasil_bc.head()

,Num Day,Tanggal,Mean Suhu Udara,ET Total
0,day 5,2024-10-26,30.90750,5.998712
1,day 6,2024-10-27,31.77875,6.106921
2,day 7,2024-10-28,32.63000,6.212646
3,day 8,2024-10-29,32.22250,6.162035
4,day 9,2024-10-30,31.17125,6.031469


# ET Model Hargreaves-Samani

In [30]:
hasil_hs_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Hargreaves-Samani
    radiasi_ekstrateresterial, et_model_hs, et_model_hs_modification = hs_model(t_max = max_suhu_udara, 
                                                                                t_min = min_suhu_udara, 
                                                                                t_avg = mean_suhu_udara, 
                                                                                latitude = var.latitude, 
                                                                                input_date = tanggal_data)
    # print(f"ET Hargreaves-Samani: {et_model_hs}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_hs_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": et_model_hs,
    })

# Buat DataFrame dari hasil
df_hasil_hs = pd.DataFrame(hasil_hs_model)

In [31]:
df_hasil_hs.to_csv('hasil/Estimasi ET Hargreaves-Samani.csv', index=False)
df_hasil_hs.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Radiasi Extrateresterial,ET Total
0,day 5,2024-10-26,33.07,25.59,29.330,15.792283,4.681888
1,day 6,2024-10-27,34.18,25.99,30.085,15.798661,4.979544
2,day 7,2024-10-28,35.19,26.08,30.635,15.804507,5.314070
3,day 8,2024-10-29,34.46,26.55,30.505,15.809836,4.940098
4,day 9,2024-10-30,32.96,26.74,29.850,15.814667,4.322609


# ET Model Penman-Monteith

In [32]:
hasil_pm_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    kecepatan_angin = np.array(df_data_sekunder['kecepatan_angin'])
    kelembapan_relatif = np.array(df_data_sekunder['kelembapan_relatif'])
    
    # Menghitung rata-rata kecepatan angin dan kelembapan relatif
    mean_kecepatan_angin = np.mean(kecepatan_angin)
    # print(f"Rata-rata kecepatan angin: {mean_kecepatan_angin}")
    mean_kelembapan_relatif = np.mean(kelembapan_relatif)
    # print(f"Rata-rata kelembapan relatif: {mean_kelembapan_relatif}")

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")
    
    # Menghitung ET dengan model Penman-monteith
    radiasi_ekstrateresterial, et_model_pm, total_et_model_pm = pm_model(RH = mean_kelembapan_relatif, 
                                                                         u_2 = kecepatan_angin, 
                                                                         T_hr = suhu_udara, 
                                                                         T_avg = mean_suhu_udara, 
                                                                         T_max = max_suhu_udara, 
                                                                         T_min = min_suhu_udara, 
                                                                         altitude = var.altitude, 
                                                                         latitude = var.latitude, 
                                                                         input_date = tanggal_data)
    # print(f"ET Penman-monteith {sheet_name}: {total_et_model_pm}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_pm_model.append({
        "Num Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Mean Kecepatan Angin": mean_kecepatan_angin,
        "Mean Kelembapan Relatif": mean_kelembapan_relatif,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": total_et_model_pm
    })

# Buat DataFrame dari hasil
df_hasil_pm = pd.DataFrame(hasil_pm_model)

In [33]:
df_hasil_pm.to_csv('hasil/Estimasi ET Penman-Monteith.csv', index=False)
df_hasil_pm.head()

,Num Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Mean Kecepatan Angin,Mean Kelembapan Relatif,Radiasi Extrateresterial,ET Total
0,day 5,2024-10-26,33.07,25.59,29.330,3.80000,48.86750,15.792283,4.104162
1,day 6,2024-10-27,34.18,25.99,30.085,2.74875,45.59375,15.798661,3.876762
2,day 7,2024-10-28,35.19,26.08,30.635,2.78875,42.54750,15.804507,4.293068
3,day 8,2024-10-29,34.46,26.55,30.505,3.95125,44.36125,15.809836,4.737096
4,day 9,2024-10-30,32.96,26.74,29.850,4.02250,50.61750,15.814667,3.990299
